<a href="https://colab.research.google.com/github/Steboss/NLP_on_TPU/blob/master/fairyTales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [2]:
!mkdir fairyTales

mkdir: cannot create directory ‘fairyTales’: File exists


In [0]:
!mv ../*.txt fairyTales/.

In [0]:
!pip install -q tensorflow==1.13.1

In [0]:
import numpy as np
import pandas as pd
import os
import glob 

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
import time
import datetime
import seaborn as sbn
from tqdm import tqdm_notebook

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

import tensorflow.keras.utils as ku 
import numpy as np 


import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)

In [5]:
tf.__version__

'1.13.1'

In [6]:
wholedataset = [] 

allfiles = glob.glob("fairyTales/*.txt")
counter = 0 

for ifile in allfiles:

    print(ifile)
    reader = open(ifile,"r").readlines()
    for line in reader:
        #strip and lower 
        lowercase = line.lower()
        stripcase = lowercase.strip()
        nonewlines = stripcase.split("\n")
        for nonewline in nonewlines:
            if nonewline=="":
                continue
            else:
                wholedataset.append(nonewline)
        

    counter+=1

    if counter==1:
        break
        
    

fairyTales/animated-architecture-modern-mag.txt


In [0]:
tokenizer=Tokenizer()

tokenizer.fit_on_texts(wholedataset)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in wholedataset:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [8]:
model = Sequential()
#100 is the embedding dimension, we can increase it
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 213, 100)          1113700   
_________________________________________________________________
bidirectional (Bidirectional (None, 213, 300)          301200    
_________________________________________________________________
dropout (Dropout)            (None, 213, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 5568)              562368    
_________________________________________________________________
dense_1 (Dense)      

In [9]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.88.121.234:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 5206806519564898940)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3940979866880049306)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17519680567430445684)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

In [0]:
%%time
#def train_gen(batch_size):
#  while True:
#    offset = np.random.randint(0, x_train.shape[0] - batch_size)
#    yield x_train[offset:offset+batch_size], y_train[offset:offset + batch_size]
    

tpu_model.fit(predictors, label, epochs=100, verbose=1)
#tpu_model.fit_generator(
#    train_gen(512),
#    epochs=15,
#    steps_per_epoch=100,
#    validation_data=(x_valid, y_valid)
#)


#number of eophcs can be increased
#history = model.fit(predictors, label, epochs=2, verbose=1)

Epoch 1/100
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 213), dtype=tf.float32, name='embedding_input_10'), TensorSpec(shape=(4, 11137), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for embedding_input
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 7.739359378814697 secs
INFO:tensorflow:Setting weights on TPU model.
 16800/127260 [==>...........................] - ETA: 47:41 - loss: 7.1055 - acc: 0.0322